Webscraping using selenium:

Selenium is necessary when scraping websites that rely on JavaScript to load content dynamically. It interacts with the browser directly, so it can wait for JavaScript to execute and for elements to appear on the page.

In [1]:
!pip install pandas
!pip install requests
!pip install bs4
!pip install html5lib 
!pip install lxml
!pip install plotly


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 3.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
## url = "https://finance.yahoo.com/quote/7010.SR/history/"

In [28]:


stc_html_data = requests.get("https://finance.yahoo.com/quote/7010.SR/history/", timeout=300).text
print(stc_html_data)

<!DOCTYPE html>
  <html lang="en-us"><head>
  <meta http-equiv="content-type" content="text/html; charset=UTF-8">
      <meta charset="utf-8">
      <title>Yahoo</title>
      <meta name="viewport" content="width=device-width,initial-scale=1,minimal-ui">
      <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
      <style>
  html {
      height: 100%;
  }
  body {
      background: #fafafc url(https://s.yimg.com/nn/img/sad-panda-201402200631.png) 50% 50%;
      background-size: cover;
      height: 100%;
      text-align: center;
      font: 300 18px "helvetica neue", helvetica, verdana, tahoma, arial, sans-serif;
  }
  table {
      height: 100%;
      width: 100%;
      table-layout: fixed;
      border-collapse: collapse;
      border-spacing: 0;
      border: none;
  }
  h1 {
      font-size: 42px;
      font-weight: 400;
      color: #400090;
  }
  p {
      color: #1A1A1A;
  }
  #message-1 {
      font-weight: bold;
      margin: 0;
  }
  #message-2 {
      display: 

In [29]:
soup = BeautifulSoup(stc_html_data, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-us">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta charset="utf-8"/>
  <title>
   Yahoo
  </title>
  <meta content="width=device-width,initial-scale=1,minimal-ui" name="viewport"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <style>
   html {
      height: 100%;
  }
  body {
      background: #fafafc url(https://s.yimg.com/nn/img/sad-panda-201402200631.png) 50% 50%;
      background-size: cover;
      height: 100%;
      text-align: center;
      font: 300 18px "helvetica neue", helvetica, verdana, tahoma, arial, sans-serif;
  }
  table {
      height: 100%;
      width: 100%;
      table-layout: fixed;
      border-collapse: collapse;
      border-spacing: 0;
      border: none;
  }
  h1 {
      font-size: 42px;
      font-weight: 400;
      color: #400090;
  }
  p {
      color: #1A1A1A;
  }
  #message-1 {
      font-weight: bold;
      margin: 0;
  }
  #message-2 {
      display: inline-b

In [30]:
table = soup.find("table")
if table:
    tbody = table.find("tbody")
    if tbody:
        rows = tbody.find_all('tr')
        print(f"Number of rows found: {len(rows)}")
    else:
        print("No tbody found")
else:
    print("No table found")


Number of rows found: 1


In [31]:
stc_data = pd.DataFrame(columns=["Date", "Open", "High", "Low", "Close", "Volume"])

In [36]:
# First we isolate the body of the table which contains all the information
# Then we loop through each row and find all the column values for each row
for row in soup.find("tbody").find_all('tr'):
    col = row.find_all("td")
    if len(col) < 7:  # Ensure there are enough columns
        continue  # Skip if there aren't enough columns
    date = col[0].text
    Open = col[1].text
    high = col[2].text
    low = col[3].text
    close = col[4].text
    adj_close = col[5].text
    volume = col[6].text
    
    # Finally we append the data of each row to the table
    stc_data = pd.concat([stc_data,pd.DataFrame({"Date":[date], "Open":[Open], "High":[high], "Low":[low], "Close":[close], "Adj Close":[adj_close], "Volume":[volume]})], ignore_index=True)    

In [37]:
stc_data.head()

,Date,Open,High,Low,Close,Volume


it seems that the website uses Javascript. So using selenium library to scrape

In [21]:
!pip install selenium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 5.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [44]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options  # Import Options
import pandas as pd

# Set the path to your ChromeDriver
chrome_driver_path = './Selenium.WebDriver.ChromeDriver.130.0.6723.600-beta/driver/mac64/chromedriver' 

# Set up Chrome options for headless mode
chrome_options = Options()
chrome_options.add_argument("--headless")  # Enable headless mode

# Set up the Service
service = Service(chrome_driver_path)

# Initialize the WebDriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Go to the Yahoo Finance historical data page
url = "https://finance.yahoo.com/quote/7010.SR/history?p=7010.SR"
driver.get(url)

# Wait for a specific element to be loaded
try:
    element_present = EC.presence_of_element_located((By.CSS_SELECTOR, "h1[data-reactid='7']"))  # Example selector
    WebDriverWait(driver, 600).until(element_present)  # Increased timeout
except TimeoutException:
    print("Timed out waiting for page to load")

    

# # Scrape the data
# stc_data = pd.DataFrame(columns=["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"])

# # Find the rows of the table
# rows = driver.find_elements(By.CSS_SELECTOR, 'table tbody tr')

# for row in rows:
#     cols = row.find_elements(By.TAG_NAME, "td")
#     if len(cols) == 7:  # Make sure there are enough columns
#         data = [col.text for col in cols]
#         stc_data = stc_data.append({
#             "Date": data[0],
#             "Open": data[1],
#             "High": data[2],
#             "Low": data[3],
#             "Close": data[4],
#             "Adj Close": data[5],
#             "Volume": data[6]
#         }, ignore_index=True)
# this code didn't work since append() has been deprecated in recent versions of pandas, 
# starting from version 2.0. To resolve this issue, you can use the pd.concat() function, 
# which provides similar functionality.

# Scrape the data
data_rows = []

# Find the rows of the table
rows = driver.find_elements(By.CSS_SELECTOR, 'table tbody tr')

for row in rows:
    cols = row.find_elements(By.TAG_NAME, "td")
    if len(cols) == 7:  # Make sure there are enough columns
        data = [col.text for col in cols]
        data_rows.append({
            "Date": data[0],
            "Open": data[1],
            "High": data[2],
            "Low": data[3],
            "Close": data[4],
            "Adj Close": data[5],
            "Volume": data[6]
        })

# Convert the list of dictionaries to a DataFrame
stc_data = pd.DataFrame(data_rows)

# Display the DataFrame
print(stc_data)


# Close the WebDriver
driver.quit()

# Display the DataFrame
print(stc_data)


Timed out waiting for page to load
             Date   Open   High    Low  Close Adj Close     Volume
0     Nov 3, 2024  42.15  42.45  42.00  42.05     42.05  2,100,138
1    Oct 31, 2024  42.40  42.60  42.15  42.15     42.15  3,702,138
2    Oct 30, 2024  42.20  42.45  42.15  42.45     42.45  2,712,671
3    Oct 29, 2024  42.45  42.50  42.10  42.20     42.20  2,244,386
4    Oct 28, 2024  42.50  42.60  42.30  42.50     42.50  2,194,105
..            ...    ...    ...    ...    ...       ...        ...
244   Nov 9, 2023  38.20  38.50  38.05  38.20     36.09  3,640,149
245   Nov 8, 2023  37.95  38.20  37.75  38.20     36.09  2,800,877
246   Nov 7, 2023  37.95  37.95  37.50  37.95     35.85  2,334,035
247   Nov 6, 2023  38.10  38.20  37.70  37.95     35.85  2,907,508
248   Nov 5, 2023  38.60  38.60  38.20  38.35     35.85  2,785,448

[249 rows x 7 columns]
             Date   Open   High    Low  Close Adj Close     Volume
0     Nov 3, 2024  42.15  42.45  42.00  42.05     42.05  2,100,138
1  

In [45]:
stc_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,"Nov 3, 2024",42.15,42.45,42.00,42.05,42.05,"2,100,138"
1,"Oct 31, 2024",42.40,42.60,42.15,42.15,42.15,"3,702,138"
2,"Oct 30, 2024",42.20,42.45,42.15,42.45,42.45,"2,712,671"
3,"Oct 29, 2024",42.45,42.50,42.10,42.20,42.20,"2,244,386"
4,"Oct 28, 2024",42.50,42.60,42.30,42.50,42.50,"2,194,105"
